### The OpenMDAO way
---
#### Create a `Component` to represent the Rosenbrock function

In [1]:
import numpy as np
import openmdao.api as om
    
import time
    
ORDER = 6  # dimension of problem
span = 2   # upper and lower limits

class RosenbrockComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', np.zeros(ORDER))
        self.add_output('y', 0.0)

    def compute(self, inputs, outputs):
        x = inputs['x']

        n = len(x)
        s = 0
        for i in range(n - 1):
            s += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2
        
        print('x:', x, 'y:', s)
        outputs['y'] = s

#### Create a model by putting the `Component` in a `Group` and specifying the design variables and objective

In [2]:
my_model = om.Group()
my_model.add_subsystem('rosenbrock', RosenbrockComp())
my_model.add_design_var('rosenbrock.x', lower=-span * np.ones(ORDER), upper=span * np.ones(ORDER))
my_model.add_objective('rosenbrock.y')

#### Create a `Problem` to optimize the model using your choice of optimizers as the `Driver`

In [ ]:
p = om.Problem(model=my_model, driver=om.DifferentialEvolutionDriver(max_gen=500))
p.setup()

start = time.time()
fail = p.run_driver()
elapsed = time.time() - start

assert fail is False

x: [ 0.54784675 -0.92085314 -1.8361059  -1.93388946  1.25308096  1.65102231] y: 4323.699018126374
x: [ 0.4265431   0.91798624  0.17449997  1.7402897   1.26341422 -1.989046  ] y: 1989.8160319602205
x: [ 1.42961711 -1.8656577   0.91862179 -1.29737752  1.45271569  0.16584488] y: 3040.6164766288707
x: [-0.80115244 -0.30925112 -1.88672132 -1.50286689  0.68249766  0.58875805] y: 3315.989529399289
x: [ 0.46154045 -0.46528978  1.98883974  1.92334136  0.74216794  0.60183711] y: 1652.1141550465832
x: [ 0.75378692 -0.4443143  -1.45961398  0.88595336  0.10141729 -0.7590325 ] y: 646.893453298103
x: [-0.05665856  1.55795134  1.73617406 -0.56881921  0.28611932 -0.71252244] y: 1641.535775625033
x: [ 0.37720012 -0.6483551  -0.433524    1.56109741 -1.09136963  0.49274858] y: 1627.5514864802874
x: [-1.66393863  1.33057659  1.14839323 -1.04252223  1.50593692 -1.76572786] y: 2459.2893079203045
x: [-0.65553176 -1.39888213 -0.19864253  1.18529708 -1.07743116 -1.7919148 ] y: 2432.6779137436342
x: [-0.38179264

#### The global minimum is found at [1, 1, 1, ...]

In [ ]:
print('xopt:', p['rosenbrock.x'])
print('fopt:', p['rosenbrock.y'])
print('iterations:', p.driver.iter_count)
print('elapsed time:', elapsed)




### The `pycma` way
---
#### Using the functional interface

In [ ]:
import cma

def rosenbrock(x):
    n = len(x)
    s = 0
    for i in range(n - 1):
        s += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2
    return s

# NOTE: the Rosenbrock function is also available in the "fitness functions" library
# rosenbrock = cma.ff.rosen

# find global minimum via the fmin function
start = time.time()
res = cma.fmin(rosenbrock, ORDER * [-1], 0.01,
               options={'ftarget':1e-6, 'verb_time':0, 'verb_disp':100, 'seed':3},
               restarts=3)
elapsed = time.time() - start

#### The global minimum is found at [1, 1, 1, ...]

In [ ]:
print('xopt:', res[0])
print('fopt:', res[1])
print('evalsopt:', res[2])
print('evals:', res[3])
print('iterations:', res[4])
print('elapsed time:', elapsed)

### Use `pycma` as top level interface over OpenMDAO
---
#### Wrap an OpenMDAO `Component` for use with the `pycma` functional interface

In [ ]:
comp = RosenbrockComp()

def rosenbrock(x):
    inputs = {'x': x}
    outputs = {}
    comp.compute(inputs, outputs)
    return outputs['y']
    
start = time.time()
res = cma.fmin(rosenbrock, ORDER * [-1], 0.01,
               options={'ftarget':1e-6, 'verb_time':0, 'verb_disp':100, 'seed':3},
               restarts=3)
elapsed = time.time() - start

#### The global minimum is found at [1, 1, 1, ...]

In [ ]:
print('xopt:', res[0])
print('fopt:', res[1])
print('evalsopt:', res[2])
print('evals:', res[3])
print('iterations:', res[4])
print('elapsed time:', elapsed)

### Use `pycma` as top level interface over OpenMDAO
---
#### Wrap an OpenMDAO `Problem` for use with the `pycma` functional interface

In [ ]:
def rosenbrock(x):
    p['rosenbrock.x'] = x
    p.run_model()
    return p['rosenbrock.y']
    
start = time.time()
res = cma.fmin(rosenbrock, ORDER * [-1], 0.01,
               options={'ftarget':1e-6, 'verb_time':0, 'verb_disp':100, 'seed':3},
               restarts=3)
elapsed = time.time() - start

#### The global minimum is found at [1, 1, 1, ...]

In [ ]:
print('xopt:', res[0])
print('fopt:', res[1])
print('evalsopt:', res[2])
print('evals:', res[3])
print('iterations:', res[4])
print('elapsed time:', elapsed)